In [1]:
# ============================================================
# VIIRS-based urban / non-urban classifier (Delhi).
## VIIRS, or the Visible Infrared Imaging Radiometer Suite, is a type of satelite imaginey.
## It captures visible and infrared images and radiometric data of the Earth's land, atmosphere, oceans, and cryosphere.
## In this example we train the classifier with VIIRS night light data from 2015 to 2024 and use it to predict urban coverage for 2025. High night light signifies high urban density growth.
# Learning Objective :  Earth Engine, remote sensing & ML learning.
# Prerequsite : Get your free ( non commerical ) Google Cloud Project (GCP) with Google Earth Engine API here : https://developers.google.com/earth-engine/#api
# Execution : Run in Google CoLab with the GCP ID
# ============================================================

# --------------------------
# SECTION 1: Install & auth
# --------------------------
# Install Earth Engine and geemap
!pip -q install earthengine-api geemap

import ee, geemap
import os
from google.colab import userdata
from datetime import datetime

# I have saved my GCP ID as secret in Gooogle Colab field 'GCP_ID' and using it below. You can directly pass your GCP credentials below instead.
PROJECT_ID = userdata.get('GCP_ID')

# Optional: make it visible to other tools via environment variable
os.environ['GCP_ID'] = PROJECT_ID if PROJECT_ID else ''

# Authenticate Earth Engine (EE)
# In Colab this will prompt you to log in the first time.
ee.Authenticate()
# Initialize EE
ee.Initialize(project=PROJECT_ID)

# Make sure your Google account has Earth Engine access (request at signup if needed).

# --------------------------
# SECTION 2: Define study region. We have taken DELHI for this use case
# --------------------------
# Here we select Delhi from a FAO GAUL dataset and take its geometry.
## For more about FAO GAUL refer - https://www.fao.org/hih-geospatial-platform/news/detail/now-available--the-global-administrative-unit-layers-(gaul)-dataset---2024-edition/en
# This defines the area we'll sample and predict over.
DELHI = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level1') \
    .filter(ee.Filter.eq('ADM1_NAME', 'Delhi')) \
    .geometry()

# --- EXPERIMENT ---
# Try changing DELHI to another region (e.g., a different ADM1_NAME or a bounding box)
# to see how the model behaves on different cities or rural regions.


# --------------------------
# SECTION 3: Load VIIRS data sources
# --------------------------
viirs_ic = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG")
# SRTM for slope (elevation derivative)
srtm = ee.Image("USGS/SRTMGL1_003").clip(DELHI)
slope = ee.Terrain.slope(srtm).rename('slope')
# slope is a useful topographic feature
# It is an auxiliary (predictor) variable that helps the model distinguish places that are likely to be urban from places that are not — it captures topography information that nightlights alone can’t.
# Adding slope often improves classification/regression because urban development strongly depends on terrain.
# Flat or gently sloping land is far more likely to be built-up than steep slopes (hillsides, cliffs). A pixel with moderate lights and low slope is more plausibly urban than one on a steep mountain with similar radiance.


# --------------------------
# SECTION 4: Helper function
# --------------------------
# Returns a VIIRS image for the specified (year, month).
# We take median of that month (should be only one monthly image) and select avg_rad band.
def viirs_month(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')
    # Filter collection to the month and take median to reduce outliers/cloud/noise
    img = viirs_ic.filterDate(start, end).median().select('avg_rad')
    return img


# --------------------------
# SECTION 5: Build training image (2015–2024 Jan–Mar)
# --------------------------
# We create one band per year-month (for Jan, Feb, Mar each year).
# This yields a multiband image where each band is "nightlight for that month".
# In this example i have taken only Quater 1 Jan to March images for a given year. You can choose any other time frame or even entire year.
# Its advised to avoid rainy season as cloud will produce less quality satelite image
training_bands = []
band_images = []

# Loop years: 2015..2024
# We are using VIIRS satelite data that is available from 2012 onwards. If you want to analyze data prior to 2012 we need to use DMSP satelite data and combine.
for year in range(2015, 2025):  # 2025 excluded
    for month in [1, 2, 3]:
        img = viirs_month(year, month)
        band_name = f'NL_{year}_{month:02d}'
        # Rename each month's band so we can refer to them consistently during training & prediction
        band_images.append(img.rename(band_name))
        training_bands.append(band_name)

# Concatenate all bands into a single image (time series stacked into bands)
full_composite = ee.Image.cat(band_images)
features = full_composite.addBands(slope).clip(DELHI)

# --- EXPERIMENT ---
# - Try using different months (e.g., [6,7,8] for summer/monsoon) to learn seasonal effects.
# - Try using fewer or more years to see how historical depth affects model stability.


# --------------------------
# SECTION 6: Prepare labels
# --------------------------
# We need labels (urban vs not) to train a supervised classifier.
# Here we create a label by thresholding VIIRS radiance in 2024 Jan–Mar (example approach).
label_imgs = []
for month in [1, 2, 3]:
    # A simple threshold: radiance >= 30 considered 'urban'.
    label_img = viirs_month(2024, month).gte(30).rename('urban_month')
    label_imgs.append(label_img)

# Combine the three binary months by median to get a stable label (value 0 or 1)
label_composite = ee.ImageCollection(label_imgs).median().rename('urban')

# Important note:
# - This approach uses 2024 observations to label pixels. If you were trying to train a model
#   that generalizes across time, using labels derived from a single year can introduce
#   biases. A more robust approach is to get independent ground truth (e.g., census population,
#   high-resolution land-cover) or label multiple years and split train/test properly.


# --------------------------
# SECTION 7: Sample training data & train classifier
# --------------------------
# Sample the training pixels from the features (input bands + slope) and label image
training_data = features.addBands(label_composite).sample(
    region=DELHI,
    scale=1000,        # --- TRY changing scale (e.g., 500, 1000, 5000) ---
                       # scale is in meters; smaller = higher spatial resolution sampling, but heavier compute.
    numPixels=5000,    # --- CHANGE numPixels to 1000, 10000 etc to see effect on model ---
    seed=42,           # seed for reproducibility; change to see different random samples
    tileScale=4        # increase if you hit memory/timeout errors in EE sampling
)

# Train a Random Forest classifier with 100 trees
# --- Try different nTrees (e.g., 10, 50, 500) and compare results ---
rf = ee.Classifier.smileRandomForest(100).train(
    features=training_data,
    classProperty='urban',
    inputProperties=training_bands + ['slope']
)


# --------------------------
# SECTION 8: Prepare 2025 input image for prediction
# --------------------------
# The code below builds a prediction image for 2025 using the SAME band names as training.
# This is a somewhat unusual approach: we are using 2025 Jan/Feb/Mar images repeatedly
# and renaming them to match the 2015..2024 band names. The intent: create the expected
# band-stack shape for the classifier even when using data from a single prediction year.

PREDICT_YEAR = 2025
predict_imgs = []

# List of training months used earlier (same order as we used to create training_bands)
training_months = [(2015, 1), (2015, 2), (2015, 3),
                   (2016, 1), (2016, 2), (2016, 3),
                   (2017, 1), (2017, 2), (2017, 3),
                   (2018, 1), (2018, 2), (2018, 3),
                   (2019, 1), (2019, 2), (2019, 3),
                   (2020, 1), (2020, 2), (2020, 3),
                   (2021, 1), (2021, 2), (2021, 3),
                   (2022, 1), (2022, 2), (2022, 3),
                   (2023, 1), (2023, 2), (2023, 3),
                   (2024, 1), (2024, 2), (2024, 3)]

# Build prediction bands by repeatedly using PREDICT_YEAR months (1,2,3)
for i, (train_year, train_month) in enumerate(training_months):
    # We cycle through Jan (1), Feb (2), Mar (3) for the prediction year to fill slots
    month_for_2025 = (i % 3) + 1
    img = viirs_month(PREDICT_YEAR, month_for_2025)
    band_name = f'NL_{train_year}_{train_month:02d}'  # same name as training band
    predict_imgs.append(img.rename(band_name))

# Add slope band to the prediction image as well
predict_image = ee.Image.cat(predict_imgs + [slope]).clip(DELHI)

# --- NOTE ---
# - This approach assumes the classifier only needs the same *names* and number of bands,
#   not the exact historical variability. It's a hack to apply a model trained on multiyear
#   stack using only one year's imagery. Better options:
#   * Build features that are temporal summaries (mean, std) across years and use those.
#   * Train on single-year features and labels per year, then predict for target year.
#   * Use a time-aware model


# --------------------------
# SECTION 9: Classify and visualize
# --------------------------
urban_prediction = predict_image.classify(rf).rename('urban')

# Visualize with geemap Map (Colab will show an interactive map)
Map = geemap.Map(center=[28.6, 77.2], zoom=8)
Map.addLayer(urban_prediction, {
    'min': 0,
    'max': 1,
    'palette': ['black', 'red']   # 0=black (non-urban), 1=red (urban)
}, 'Urban Prediction 2025')
Map.addLayer(DELHI, {}, 'Delhi Boundary')

# Show map in notebook
Map



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.3 MB/s eta 0:00:00


Map(center=[28.6, 77.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…